In [1]:
import pandas as pd

In [2]:
import numpy as np
import sklearn as skl
import os
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
sam=pd.read_csv('sample_submission.csv')

In [5]:
ret_col=sam.loc[:,'return']

In [6]:
test[0:3]


portfolio_id      desk_id    office_id pf_category  start_date         sold  \
0   PF00001001  DSK00001001  OFF00001001           A    20040720  171831000.0   
1   PF00001004  DSK00001003  OFF00001002           B    20040723   56485000.0   
2   PF00001009  DSK00001007  OFF00001001           A    20040607   41734000.0   

  country_code  euribor_rate currency  libor_rate        bought  \
0            T       0.02074      USD    2.332216  1.715337e+08   
1            T       0.02074      USD    2.332216  5.638038e+07   
2            T       0.02074      USD    2.332216  4.166179e+07   

   creation_date indicator_code  sell_date type hedge_value status  
0       20040720            NaN   20040812    A         NaN    NaN  
1       20040723            NaN   20040719    B         NaN    NaN  
2       20040726            NaN   20040709    A         NaN    NaN

In [7]:
train.describe()

start_date          sold  euribor_rate   libor_rate        bought  \
count  9.366000e+03  9.364000e+03   9366.000000  8892.000000  9.364000e+03   
mean   2.011706e+07  1.019740e+08      0.009383     0.998066  1.018914e+08   
std    3.052996e+04  1.928066e+08      0.013298     1.457977  1.927472e+08   
min    2.004041e+07  1.100000e+00     -0.003740    -0.667095  1.099698e+00   
25%    2.010020e+07  2.763200e+07      0.000980     0.244490  2.761285e+07   
50%    2.011120e+07  5.517050e+07      0.004390     0.531983  5.512056e+07   
75%    2.014071e+07  1.144632e+08      0.013510     1.109959  1.144459e+08   
max    2.017103e+07  1.320000e+10      0.050880     6.379137  1.320000e+10   

       creation_date     sell_date       return  
count   9.366000e+03  9.366000e+03  9366.000000  
mean    2.011706e+07  2.011770e+07     0.014100  
std     3.052372e+04  3.060900e+04     0.020549  
min     2.004072e+07  2.004071e+07    -0.009600  
25%     2.010020e+07  2.010023e+07     0.003000  
50%     2.011120e+07  2.011121e+07     0.005400  
75%     2.014071e+07  2.014083e+07     0.014520  
max     2.017103e+07  2.017103e+07     0.778441

In [8]:
ids=train['portfolio_id']
test_ids=test['portfolio_id']

In [9]:
sam_id=sam['portfolio_id']

In [10]:
len(sam_id)

4801

In [11]:
k=[]
for i,j in zip(sorted(sam_id),sorted(ids)):
    if i!=j:
        k.append(j)

In [12]:
len(k)

4801

In [13]:
k[0:5]

['PF00001002', 'PF00001003', 'PF00001005', 'PF00001006', 'PF00001007']

In [14]:
sam_return=sam['return']
train_return=train['return']


In [15]:
train_return[0:5]

0    0.02496
1    0.05496
2    0.02496
3    0.02496
4    0.02496
Name: return, dtype: float64

In [16]:
print(set(train['currency']))
print(set(train['type']))
print(set(train['country_code']))
print(set(train['pf_category']))
print(set(train['hedge_value']))
print(set(train['status']))

{'EUR', 'JPY', 'GBP', 'USD', 'CHF'}
{'C', 'D', 'A', 'F', 'B', 'G', 'H', 'E'}
{'Z', 'T', 'M', 'N', 'U'}
{'C', 'D', 'A', 'B', 'E'}
{False, True, nan}
{True, nan}


In [17]:
train[0:3]

portfolio_id      desk_id    office_id pf_category  start_date         sold  \
0   PF00001002  DSK00001001  OFF00001002           B    20040720  110000000.0   
1   PF00001003  DSK00001002  OFF00001001           A    20040709  176671000.0   
2   PF00001005  DSK00001004  OFF00001001           A    20040723   56474000.0   

  country_code  euribor_rate currency  libor_rate        bought  \
0            T       0.02074      USD    2.332216  1.098097e+08   
1            N       0.02074      GBP    5.269617  1.760084e+08   
2            T       0.02074      USD    2.332216  5.637953e+07   

   creation_date indicator_code  sell_date type hedge_value status   return  
0       20040720            NaN   20040812    B         NaN    NaN  0.02496  
1       20040723            NaN   20040812    C         NaN    NaN  0.05496  
2       20040723            NaN   20040817    A         NaN    NaN  0.02496

In [18]:
train_1 = train.drop(['portfolio_id', 'desk_id','office_id', 'creation_date', 'sell_date', 'indicator_code', 'status','return'], axis=1)

test_1 = test.drop(['portfolio_id', 'desk_id','office_id', 'creation_date', 'sell_date', 'indicator_code', 'status'], axis=1)

In [19]:
train_1[0:3]

pf_category  start_date         sold country_code  euribor_rate currency  \
0           B    20040720  110000000.0            T       0.02074      USD   
1           A    20040709  176671000.0            N       0.02074      GBP   
2           A    20040723   56474000.0            T       0.02074      USD   

   libor_rate        bought type hedge_value  
0    2.332216  1.098097e+08    B         NaN  
1    5.269617  1.760084e+08    C         NaN  
2    2.332216  5.637953e+07    A         NaN

In [20]:
train_1['hedge_value'].fillna(False,inplace=True)
test_1['hedge_value'].fillna(False,inplace=True)

C:\Users\raina\AppData\Local\Temp\ipykernel_25216\539674585.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_1['hedge_value'].fillna(False,inplace=True)
C:\Users\raina\AppData\Local\Temp\ipykernel_25216\539674585.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_1['hedge_value'].fillna(False,in

In [21]:
print(set(train_1['currency']))
print(set(train_1['type']))
print(set(train_1['country_code']))
print(set(train_1['pf_category']))
print(set(train_1['hedge_value']))


{'EUR', 'JPY', 'GBP', 'USD', 'CHF'}
{'C', 'D', 'A', 'F', 'B', 'G', 'H', 'E'}
{'Z', 'T', 'M', 'N', 'U'}
{'C', 'D', 'A', 'B', 'E'}
{False, True}


In [22]:
train_1.columns

Index(['pf_category', 'start_date', 'sold', 'country_code', 'euribor_rate',
       'currency', 'libor_rate', 'bought', 'type', 'hedge_value'],
      dtype='object')

In [23]:
train_1.isnull().sum()

pf_category       0
start_date        0
sold              2
country_code      0
euribor_rate      0
currency          0
libor_rate      474
bought            2
type              0
hedge_value       0
dtype: int64

In [24]:
train_1['sold'].fillna(train_1['sold'].median(),inplace=True)
train_1['bought'].fillna(train_1['bought'].median(),inplace=True)
train_1['libor_rate'].fillna(train_1['libor_rate'].median(),inplace=True)

test_1['sold'].fillna(test_1['sold'].median(),inplace=True)
test_1['bought'].fillna(test_1['bought'].median(),inplace=True)
test_1['libor_rate'].fillna(test_1['libor_rate'].median(),inplace=True)

C:\Users\raina\AppData\Local\Temp\ipykernel_25216\2939284951.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_1['sold'].fillna(train_1['sold'].median(),inplace=True)
C:\Users\raina\AppData\Local\Temp\ipykernel_25216\2939284951.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [25]:
train_1[0:3]

pf_category  start_date         sold country_code  euribor_rate currency  \
0           B    20040720  110000000.0            T       0.02074      USD   
1           A    20040709  176671000.0            N       0.02074      GBP   
2           A    20040723   56474000.0            T       0.02074      USD   

   libor_rate        bought type  hedge_value  
0    2.332216  1.098097e+08    B        False  
1    5.269617  1.760084e+08    C        False  
2    2.332216  5.637953e+07    A        False

In [26]:
data = [['a',0],['b',1],['c',3],['d',4],['e',5]]
da=pd.DataFrame(data,['1','2','3','4','5'])
print(da)

   0  1
1  a  0
2  b  1
3  c  3
4  d  4
5  e  5


In [27]:
da[0].replace(['a','b'],['AB','CD'])

1    AB
2    CD
3     c
4     d
5     e
Name: 0, dtype: object

In [28]:
print(set(train_1['currency']))
print(set(train_1['type']))
print(set(train_1['country_code']))
print(set(train_1['pf_category']))
print(set(train_1['hedge_value']))

{'EUR', 'JPY', 'GBP', 'USD', 'CHF'}
{'C', 'D', 'A', 'F', 'B', 'G', 'H', 'E'}
{'Z', 'T', 'M', 'N', 'U'}
{'C', 'D', 'A', 'B', 'E'}
{False, True}


In [29]:
train_1[0:3]

pf_category  start_date         sold country_code  euribor_rate currency  \
0           B    20040720  110000000.0            T       0.02074      USD   
1           A    20040709  176671000.0            N       0.02074      GBP   
2           A    20040723   56474000.0            T       0.02074      USD   

   libor_rate        bought type  hedge_value  
0    2.332216  1.098097e+08    B        False  
1    5.269617  1.760084e+08    C        False  
2    2.332216  5.637953e+07    A        False

In [30]:
train_1['currency'].replace(['CHF', 'USD', 'JPY', 'GBP', 'EUR'],['10','20','30','40','50'], inplace=True)
train_1['type'].replace(['B', 'A', 'G', 'E', 'F', 'H', 'C', 'D'],['1','2','3','4','5','6','7','8'], inplace=True)
train_1['country_code'].replace(['N', 'M', 'Z', 'T', 'U'],['1','2','3','4','5'], inplace=True)
train_1['pf_category'].replace(['B', 'A', 'E', 'C', 'D'],['6','7','8','9','10'], inplace=True)
train_1['hedge_value'].replace([False, True],['0','1'], inplace=True)


test_1['currency'].replace(['CHF', 'USD', 'JPY', 'GBP', 'EUR'],['10','20','30','40','50'], inplace=True)
test_1['type'].replace(['B', 'A', 'G', 'E', 'F', 'H', 'C', 'D'],['1','2','3','4','5','6','7','8'], inplace=True)
test_1['country_code'].replace(['N', 'M', 'Z', 'T', 'U'],['1','2','3','4','5'], inplace=True)
test_1['pf_category'].replace(['B', 'A', 'E', 'C', 'D'],['6','7','8','9','10'], inplace=True)
test_1['hedge_value'].replace([False, True],['0','1'], inplace=True)

C:\Users\raina\AppData\Local\Temp\ipykernel_25216\4252903030.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_1['currency'].replace(['CHF', 'USD', 'JPY', 'GBP', 'EUR'],['10','20','30','40','50'], inplace=True)
C:\Users\raina\AppData\Local\Temp\ipykernel_25216\4252903030.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which

In [31]:
train_1[0:3]

pf_category  start_date         sold country_code  euribor_rate currency  \
0           6    20040720  110000000.0            4       0.02074       20   
1           7    20040709  176671000.0            1       0.02074       40   
2           7    20040723   56474000.0            4       0.02074       20   

   libor_rate        bought type hedge_value  
0    2.332216  1.098097e+08    1           0  
1    5.269617  1.760084e+08    7           0  
2    2.332216  5.637953e+07    2           0

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=7)
scores = cross_val_score(forest_reg, train_1, train_return, scoring='r2', cv=5)
print(scores)
print('mean r2:',np.mean(scores))

[-0.71581735  0.86862733  0.92864093  0.40066109  0.71309368]
mean r2: 0.4390411380442984


In [33]:
forest_reg = RandomForestRegressor(random_state=7)
forest_reg.fit(train_1, train_return)
preds = forest_reg.predict(test_1)

sub = pd.DataFrame({'portfolio_id': test_ids, 'return': preds})
filename = 'sub_returns.csv'
sub.to_csv(filename, index=False)